In [1]:
import os
import pandas as pd
import glob

def load_and_examine_fcc_data(data_folder_path):
    """
    Loads FCC auction data files from a specified folder and examines their headers.
    
    Args:
        data_folder_path (str): Path to the folder containing FCC auction data files
        
    Returns:
        dict: Dictionary mapping filenames to their respective headers
    """
    # Check if the folder exists
    if not os.path.exists(data_folder_path):
        raise FileNotFoundError(f"The folder {data_folder_path} does not exist")
    
    # Get all CSV files in the folder
    file_paths = glob.glob(os.path.join(data_folder_path, "*.csv"))
    
    if not file_paths:
        print(f"No CSV files found in {data_folder_path}")
        return {}
    
    headers_by_file = {}
    
    # Load each file and examine headers
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        try:
            # Read just the header row to save memory
            df = pd.read_csv(file_path, nrows=0)
            headers = list(df.columns)
            headers_by_file[file_name] = headers
            print(f"Successfully loaded headers from {file_name}")
        except Exception as e:
            print(f"Error loading {file_name}: {str(e)}")
    
    return headers_by_file

def main():
    # Path to your data folder - update this to your specific path
    data_folder = "./data"
    
    print(f"Examining FCC auction data files in {data_folder}")
    headers_dict = load_and_examine_fcc_data(data_folder)
    
    # Display headers for each file
    for file_name, headers in headers_dict.items():
        print(f"\nFile: {file_name}")
        print(f"Number of columns: {len(headers)}")
        print("Headers:")
        for i, header in enumerate(headers, 1):
            print(f"  {i}. {header}")

if __name__ == "__main__":
    main()

Examining FCC auction data files in ./data
Successfully loaded headers from results_auction108.csv
Successfully loaded headers from bidder_status_auction102.csv
Successfully loaded headers from bidder_prs_auction108.csv
Successfully loaded headers from product_status_auction108.csv
Successfully loaded headers from results_auction102.csv
Successfully loaded headers from product_status_auction102.csv
Successfully loaded headers from markets_auction102.csv
Successfully loaded headers from market_auction108.csv
Successfully loaded headers from unassigned_licenses_auction108.csv

File: results_auction108.csv
Number of columns: 13
Headers:
  1. auction_id
  2. round
  3. bidder
  4. frn
  5. market
  6. market_name
  7. category
  8. processed_demand
  9. fully_processed_flag
  10. processed_demand_detail
  11. bidding_units
  12. aggregate_demand
  13. posted_price

File: bidder_status_auction102.csv
Number of columns: 22
Headers:
  1. auction_id
  2. round
  3. bidder
  4. frn
  5. bidding

In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

class FCCAuctionDataAnalyzer:
    """
    A class for analyzing FCC auction data, particularly for Auction 108 (2.5 GHz).
    This analyzer loads, processes, and helps visualize auction data to prepare
    for predictive modeling of spectrum auction prices.
    """
    
    def __init__(self, data_folder_path):
        """
        Initialize the analyzer with the path to the data folder.
        
        Args:
            data_folder_path (str): Path to the folder containing FCC auction data files
        """
        self.data_folder = Path(data_folder_path)
        self.file_paths = {}
        self.data = {}
        self.file_patterns = {
            "announcements": "announcements*.csv",
            "round_summary": "round_summary*.csv",
            "bids": "bids*.csv",
            "results": "results*.csv",
            "results_by_license": "results_by_license*.csv",
            "unassigned_licenses": "unassigned_licenses*.csv",
            "product_status": "product_status*.csv",
            "bidder_status": "bidder_status*.csv",
            "bidder_markets": "bidder_markets*.csv",
            "bidders": "bidders*.csv",
            "markets": "markets*.csv"
        }
        
        # Check if the folder exists
        if not os.path.exists(data_folder_path):
            raise FileNotFoundError(f"The folder {data_folder_path} does not exist")
        
        # Discover available data files
        self._discover_files()
    
    def _discover_files(self):
        """
        Discover all available FCC auction data files in the data folder.
        """
        for file_type, pattern in self.file_patterns.items():
            matching_files = list(self.data_folder.glob(pattern))
            if matching_files:
                self.file_paths[file_type] = matching_files
    
    def list_available_files(self):
        """
        List all available data files found in the data folder.
        
        Returns:
            dict: Dictionary of file types and their paths
        """
        available_files = {}
        for file_type, paths in self.file_paths.items():
            available_files[file_type] = [path.name for path in paths]
        
        return available_files
    
    def load_data(self, file_types=None):
        """
        Load specified data files into memory.
        
        Args:
            file_types (list, optional): List of file types to load. 
                                         If None, loads all available files.
        
        Returns:
            dict: Dictionary of loaded data frames
        """
        if file_types is None:
            file_types = list(self.file_paths.keys())
        
        for file_type in file_types:
            if file_type in self.file_paths:
                try:
                    for path in self.file_paths[file_type]:
                        # Handle date parsing for time columns
                        if file_type == "announcements" or file_type == "round_summary":
                            self.data[file_type] = pd.read_csv(
                                path, 
                                parse_dates=["announcement_time" if file_type == "announcements" 
                                             else "start_time", "end_time"]
                            )
                        else:
                            self.data[file_type] = pd.read_csv(path)
                        
                        print(f"Successfully loaded {path.name}")
                except Exception as e:
                    print(f"Error loading {file_type}: {str(e)}")
        
        return self.data
    
    def display_file_info(self, file_types=None):
        """
        Display basic information about loaded data files.
        
        Args:
            file_types (list, optional): List of file types to display info for. 
                                         If None, displays info for all loaded files.
        """
        if file_types is None:
            file_types = list(self.data.keys())
        
        for file_type in file_types:
            if file_type in self.data:
                df = self.data[file_type]
                print(f"\n=== {file_type.upper()} ===")
                print(f"Shape: {df.shape}")
                print(f"Columns: {', '.join(df.columns)}")
                print("\nSample data:")
                print(df.head(3))
                print("\nData types:")
                print(df.dtypes)
    
    def get_column_descriptions(self, file_type):
        """
        Get descriptions of columns for a specific file type based on documentation.
        
        Args:
            file_type (str): The type of file to get column descriptions for
            
        Returns:
            dict: Dictionary mapping column names to their descriptions
        """
        # Column descriptions based on the provided documentation
        descriptions = {
            # Results by License file column descriptions
            "results_by_license": {
                "auction_id": "The FCC auction number for the auction",
                "license": "The combined license name",
                "market": "The county ID",
                "market_name": "The county name",
                "category": "The license category (C1, C2, C3)",
                "bidder": "Bidder name",
                "frn": "The bidder's FCC Registration Number",
                "bidding_credit_type": "Indicates the type of bidding credit for the bidder",
                "bidding_credit": "The bidding credit percentage (0, 15, 25)",
                "gross_license_price": "The gross price of the license",
                "net_license_price": "The net price of the license after bidding credit discount",
                "effective_bidding_credit": "Calculated as 100 times (1-(net_license_price/gross_license_price))"
            },
            # Markets file column descriptions
            "markets": {
                "auction_id": "The FCC auction number for the auction",
                "market": "The county ID",
                "market_name": "The county name",
                "census_id": "The FIPS ID for the county",
                "population": "Population of the county",
                "small_market_indicator": "A flag indicating if the market is subject to the small market bidding credit cap"
            },
            # Add other file types as needed...
        }
        
        if file_type in descriptions:
            return descriptions[file_type]
        else:
            return {}
    
    def analyze_final_prices(self):
        """
        Analyze the final prices from the results_by_license file.
        """
        if "results_by_license" not in self.data:
            print("Results by license data not loaded. Please load it first.")
            return
        
        df = self.data["results_by_license"]
        
        # Basic price statistics
        print("=== PRICE ANALYSIS ===")
        print("\nGross License Price Statistics:")
        print(df['gross_license_price'].describe())
        
        # Price distribution by category
        print("\nGross License Price by Category:")
        print(df.groupby('category')['gross_license_price'].describe())
        
        # Create a price histogram
        plt.figure(figsize=(10, 6))
        sns.histplot(df['gross_license_price'], bins=30)
        plt.title('Distribution of Gross License Prices')
        plt.xlabel('Price ($)')
        plt.ylabel('Frequency')
        plt.show()
        
        # Create boxplot by category
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='category', y='gross_license_price', data=df)
        plt.title('License Prices by Category')
        plt.xlabel('License Category')
        plt.ylabel('Gross Price ($)')
        plt.show()
    
    def merge_price_and_market_data(self):
        """
        Merge the results_by_license data with markets data to enable analysis of
        price vs population and other market characteristics.
        
        Returns:
            DataFrame: Merged data with license prices and market characteristics
        """
        if "results_by_license" not in self.data or "markets" not in self.data:
            print("Required data not loaded. Please load results_by_license and markets data first.")
            return None
        
        # Merge the datasets
        merged_df = pd.merge(
            self.data["results_by_license"],
            self.data["markets"],
            on=["auction_id", "market", "market_name"],
            how="left"
        )
        
        # Create price per population metric
        merged_df['price_per_pop'] = merged_df['gross_license_price'] / merged_df['population']
        
        return merged_df
    
    def prepare_model_dataset(self):
        """
        Prepare a dataset suitable for the random forest model to predict auction prices.
        
        Returns:
            DataFrame: Prepared dataset with features and target variable
        """
        # First, merge price and market data
        merged_df = self.merge_price_and_market_data()
        if merged_df is None:
            return None
        
        # Create additional features that might be useful for prediction
        model_df = merged_df.copy()
        
        # Convert categorical variables to dummy variables
        model_df = pd.get_dummies(model_df, columns=['category', 'small_market_indicator'])
        
        # Log transform the population (common in econometric models)
        model_df['log_population'] = np.log1p(model_df['population'])
        
        # Define target variable
        model_df['target_price'] = model_df['gross_license_price']
        
        # Select features for the model
        feature_cols = [
            'population', 'log_population', 
            'category_C1', 'category_C2', 'category_C3',
            'small_market_indicator_Y', 'small_market_indicator_N'
        ]
        
        # Return both the full dataset and a subset with just the features and target
        model_features = model_df[feature_cols]
        model_target = model_df['target_price']
        
        return {
            'full_data': model_df,
            'features': model_features,
            'target': model_target
        }

def main():
    # Update this path to your specific data folder location
    data_folder = "./data"
    
    # Initialize the analyzer
    analyzer = FCCAuctionDataAnalyzer(data_folder)
    
    # List available files
    available_files = analyzer.list_available_files()
    print("Available files:")
    for file_type, files in available_files.items():
        print(f"  {file_type}: {', '.join(files)}")
    
    # Load all available data
    analyzer.load_data()
    
    # Display information about loaded files
    analyzer.display_file_info()
    
    # Analyze final prices (if results_by_license data is available)
    if "results_by_license" in analyzer.data:
        analyzer.analyze_final_prices()
    
    # Prepare dataset for modeling
    model_data = analyzer.prepare_model_dataset()
    if model_data is not None:
        print("\n=== MODEL DATASET PREVIEW ===")
        print(model_data['full_data'].head())
        print("\nFeatures shape:", model_data['features'].shape)
        print("Target shape:", model_data['target'].shape)

if __name__ == "__main__":
    main()